#Building a training data set

The aim of this script is to read data from the MBED accelerometer and a pressure sensor to build a training data set.

First, individual events will be recorded using the accelerometer and pressure sensor. The events will be saved in the matrix ${\bf e_{M}}$. The matrix will have four columns, acceleration in $x_{N}, y_{N}, z_{N}$ and pressure $p_{N}$, and it will have $N$ rows.

$$
{\rm event~matrix:~~~} {\bf e_{M}}~=~\left[
\begin{matrix}
x_{1} & y_{1} & z_{1} & p_{1}\\
x_{2} & y_{2} & z_{2}& p_{2}\\
\vdots & \vdots & \vdots & \vdots\\ 
x_{N} & y_{N} & z_{N} & p_{N}\\
\end{matrix}
\right]
$$

Each matrix will be exported into a file with the header indicating the type of event, which are encoded as follows:

```
walk = 0
stand = 1
sit down = 2
lie down = 3
fall = 4
```

In [ ]:
import serial
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#import mlpy

#data = open('fall_data_set.dat','w')
#headerLine = ("X \t Y \t Z \n")
#data.write(headerLine)

ser = serial.Serial('/dev/tty.usbmodem1412',9600,timeout=1,parity=serial.PARITY_EVEN,rtscts=1)
ser.flushInput()

n_timepoints = 50
n_parameters = 3

e_M = np.zeros(shape=(n_timepoints,n_parameters))
it = 0

for j in range(0,10):
    
try:
    while it < n_timepoints:
        #print it
        try:
            line = ser.readline()   
            line = line.strip() #removes trailing whitespaces at start&end of string   
            print line
            buf = np.array([float(val) for val in line.split()])
            if(len(buf)==n_parameters):
                #print buf
                e_M[it] = buf
                it += 1
        except:
               continue 
except:
       pass



plt.plot(e_M)
plt.show()

np.savetxt('event50.txt',e_M,header='0')

ser.flushInput()
ser.close()

Now, a feature matrix ${\bf X}$ is constructed in which each line contains one event $e_{M}$. The element $a_{M}$ in the accompanying label vector $A$ will indicate what type of event $e_{M}$ is, *e.g.* ``fall`` or ``sit down``.

$$
{\rm feature~matrix:~~~} {\bf X}~=~\left[
\begin{matrix}
e_{1}\\
e_{2}\\
\vdots\\ 
e_{M}\\
\end{matrix}
\right]
$$

$$
{\rm label~vector:~~~} {\bf A}~=~ [a_1, a_2, \cdots a_M]
$$

In [6]:
import numpy as np

n_events = 409
n_parameters = 3
n_timepoints = 50

X = np.zeros((n_events,n_parameters,n_timepoints), dtype=float)
A = np.zeros(n_events, dtype=int)

print X.shape

for i in range(n_events):
    
    #read files and create feature matrix X
    filename = 'AccData/event'+str(i+50)+'.txt'
    e_M = np.loadtxt(filename, dtype=float)
    X[i,:,:] = e_M.T
    
    #read file headers and create label vector A
    f = open(filename,'r')
    line = f.readline()
    f.close()
    a_M = line.split()[1]
    A[i] = a_M
    
print X
print A

np.save('AccData/X.npy',X)
np.save('AccData/A.npy',A)

(409, 3, 50)
[[[  5.80000000e-02   5.70000000e+01   5.40000000e-02 ...,   5.30000000e-02
     5.00000000e-02   5.50000000e-02]
  [  9.10000000e-02   9.00000000e-02   9.10000000e-02 ...,   8.80000000e-02
     1.35000000e-01   9.00000000e-02]
  [  9.99000000e-01   1.00000000e+00   9.98000000e-01 ...,   9.99000000e-01
     9.78000000e-01   9.94000000e-01]]

 [[  8.70000000e-02   3.20000000e-02   1.80000000e-02 ...,   2.17000000e-01
     2.10000000e-02   1.44000000e-01]
  [  1.10200000e+00   9.71000000e-01   9.71000000e-01 ...,   9.26000000e-01
     9.74000000e-01   9.78000000e-01]
  [  3.25000000e-01   2.71000000e-01   2.59000000e-01 ...,   2.28000000e-01
     2.61000000e-01   2.99000000e-01]]

 [[  1.15000000e-01   3.90000000e-02   2.72000000e-01 ...,   1.49000000e-01
     3.30000000e-02   6.60000000e-02]
  [  1.34000000e+00   1.05800000e+00   7.26000000e-01 ...,   1.00800000e+00
     1.01400000e+00   8.61000000e-01]
  [  3.67000000e-01   3.91000000e-01   1.27000000e-01 ...,   1.87000000